# Tâche 3 - Corriger des proverbes avec des *transformers*

Tout comme pour le premier travail pratique, l'objectif de cette tâche est de corriger des proverbes. Cependant vous devrez choisir et utiliser des modèles transformers (2) pour identifier le mot à remplacer et choisir le meilleur mot à insérer en fonction du contexte du proverbe. 


Consignes : 
- Corriger un proverbe consiste à remplacer un verbe par l’un des mots proposés dans la liste. 
- Choix des modèles transformer: Vous devrez choisir 2 modèles, un pour faire l’analyse grammaticale (*POS tagging*) et l'autre pour choisir le mot à insérer dans le proverbe. Deux contraintes : a) pas de LLMs, et b) utilisez des modèles de HuggingFace qui peuvent traiter le français (ou qui sont multilingues). 
- Entraînement des modèles : Vous pouvez utiliser les modèles préentraînés sans modification. Me consulter en cas de doute.  
- Segmentation : Ne pas segmenter un proverbe en sous-phrases. 
- Tokenisation et plongements de mots : Ceux des modèles utilisés. 
- Prétraitement et normalisation : Rien à faire. 
- Code : Utilisez le fichier ***t3_corriger_proverbes.ipynb*** (ce fichier) pour mener vos expérimentations. 
- Données : Évaluez la performance à l’aide du fichier ***data/t3_test_proverbes.json*** et faire l'analyse de vos résultats. 
- Question : Comment se compare l’approche retenue pour cette tâche par rapport à celle du Travail pratique #1?

Vous pouvez ajouter au *notebook* toutes les cellules dont vous avez besoin pour votre code, vos explications ou la présentation de vos résultats. Vous pouvez également ajouter des sous-sections (par ex. des sous-sections 1.1, 1.2 etc.) si cela améliore la lisibilité.

Notes :
- Évitez les bouts de code trop longs ou trop complexes. Par exemple, il est difficile de comprendre 4-5 boucles ou conditions imbriquées. Si c'est le cas, définissez des sous-fonctions pour refactoriser et simplifier votre code. 
- Expliquez sommairement votre démarche.
- Expliquez les choix que vous faites au niveau de la programmation et des modèles (si non trivial).
- Analyser vos résultats. Indiquez ce que vous observez, si c'est bon ou non, si c'est surprenant, etc. 
- Une analyse quantitative et qualitative d'erreurs est intéressante et permet de mieux comprendre le comportement d'un modèle.

## Section 1 - Lecture du fichier de test 

Le fichier de test ***./data/t3_test_proverbes.json*** contient les proverbes modifiés, la liste de mots candidats et la bonne version du proverbe. 

In [1]:
import json

def load_tests(filename):
    with open(filename, 'r', encoding='utf-8') as fp:
        test_data = json.load(fp)
    return test_data

In [2]:
test_fn = './data/t3_test_proverbes.json'  # Le fichier de test = À modifier selon votre configuration

In [3]:
tests = load_tests(test_fn)

In [21]:
import pandas as pd

def get_dataframe(test_proverbs):
    return pd.DataFrame.from_dict(test_proverbs, orient='columns', dtype=None, columns=None)

df = get_dataframe(tests)
df

,Masked,Word_list,Proverb
0,a beau mentir qui part de loin,"[vient, revient]",a beau mentir qui vient de loin
1,a beau dormir qui vient de loin,"[partir, mentir]",a beau mentir qui vient de loin
2,l’occasion forge le larron,"[fait, occasion]",l’occasion fait le larron
3,"endors-toi, le ciel t’aidera","[bouge, aide]","aide-toi, le ciel t’aidera"
4,"aide-toi, le ciel t’aura","[aidera, aide]","aide-toi, le ciel t’aidera"
5,"ce que femme dit, dieu le veut","[dit, veut]","ce que femme veut, dieu le veut"
6,"ce que femme veut, dieu le souhaite","[dit, veut]","ce que femme veut, dieu le veut"
7,bien mal acquis ne sait jamais,"[profite, fait]",bien mal acquis ne profite jamais
8,bon ouvrier ne déplace pas ses outils,"[fait, querelle]",bon ouvrier ne querelle pas ses outils
9,"pour le fou, c’était tous les jours fête","[est, es]","pour le fou, c’est tous les jours fête"


## Section 2 - Code pour repérer les mots qui pourraient être remplacés dans un proverbe modifié

Expliquez ici comment vous procédez pour identifier les mots d'un proverbe qui pourraient faire l'objet d'une substitution.  



In [5]:
import torch
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("gilf/french-camembert-postag-model")
model = AutoModelForTokenClassification.from_pretrained("gilf/french-camembert-postag-model")


pos_tagger = pipeline("ner", model=model, tokenizer = tokenizer, grouped_entities=True)


/Users/irina/anaconda/envs/NLP_TP2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at gilf/french-camembert-postag-model were not used when initializing CamembertForTokenClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing CamembertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is 

In [6]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import torch

tokenizer = AutoTokenizer.from_pretrained("gilf/french-camembert-postag-model")
model = AutoModelForTokenClassification.from_pretrained("gilf/french-camembert-postag-model")

pos_tagger = pipeline("ner", model="gilf/french-camembert-postag-model", aggregation_strategy="simple")

Some weights of the model checkpoint at gilf/french-camembert-postag-model were not used when initializing CamembertForTokenClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing CamembertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at gilf/french-camembert-postag-model were not used when initializing CamembertForTokenClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing CamembertForTokenClassification from the checkpoint of a m

In [7]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import torch

tokenizer = AutoTokenizer.from_pretrained("gilf/french-camembert-postag-model")
model = AutoModelForTokenClassification.from_pretrained("gilf/french-camembert-postag-model")

pos_tagger = pipeline("ner", model=model, tokenizer = tokenizer, aggregation_strategy="simple")

def get_verbs(phrase):
    tags = pos_tagger(phrase)
    print(tags)
    verbes = [tag['word'] for tag in tags if tag['entity_group'] in ['V', 'VIMP', 'VINF', 'V', 'VPP', 'VR', 'VS' ]]
    return verbes

df['verbes_in_masked'] = df['Masked'].apply(get_verbs)
print(df.head(10))




Some weights of the model checkpoint at gilf/french-camembert-postag-model were not used when initializing CamembertForTokenClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing CamembertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity_group': 'V', 'score': np.float32(0.9944412), 'word': 'a', 'start': 0, 'end': 1}, {'entity_group': 'ADJ', 'score': np.float32(0.589934), 'word': 'beau', 'start': 1, 'end': 6}, {'entity_group': 'VINF', 'score': np.float32(0.96936554), 'word': 'mentir', 'start': 6, 'end': 13}, {'entity_group': 'PROREL', 'score': np.float32(0.9933494), 'word': 'qui', 'start': 13, 'end': 17}, {'entity_group': 'V', 'score': np.float32(0.99252725), 'word': 'part', 'start': 17, 'end': 22}, {'entity_group': 'P', 'score': np.float32(0.9994779), 'word': 'de', 'start': 22, 'end': 25}, {'entity_group': 'ADV', 'score': np.float32(0.9980952), 'word': 'loin', 'start': 25, 'end': 30}]
[{'entity_group': 'V', 'score': np.float32(0.99494654), 'word': 'a', 'start': 0, 'end': 1}, {'entity_group': 'ADJ', 'score': np.float32(0.575452), 'word': 'beau', 'start': 1, 'end': 6}, {'entity_group': 'VINF', 'score': np.float32(0.9855913), 'word': 'dormir', 'start': 6, 'end': 13}, {'entity_group': 'PROREL', 'score': np.float3

,Masked,Word_list,Proverb,verbes_in_masked
0,a beau mentir qui part de loin,"[vient, revient]",a beau mentir qui vient de loin,"[a, mentir, part]"
1,a beau dormir qui vient de loin,"[partir, mentir]",a beau mentir qui vient de loin,"[a, dormir, vient]"
2,l’occasion forge le larron,"[fait, occasion]",l’occasion fait le larron,[forge]
3,"endors-toi, le ciel t’aidera","[bouge, aide]","aide-toi, le ciel t’aidera","[en, dors-, aidera]"
4,"aide-toi, le ciel t’aura","[aidera, aide]","aide-toi, le ciel t’aidera","[aide, -, aura]"
5,"ce que femme dit, dieu le veut","[dit, veut]","ce que femme veut, dieu le veut","[dit, veut]"
6,"ce que femme veut, dieu le souhaite","[dit, veut]","ce que femme veut, dieu le veut","[veut, souhaite]"
7,bien mal acquis ne sait jamais,"[profite, fait]",bien mal acquis ne profite jamais,"[acquis, sait]"
8,bon ouvrier ne déplace pas ses outils,"[fait, querelle]",bon ouvrier ne querelle pas ses outils,[déplace]
9,"pour le fou, c’était tous les jours fête","[est, es]","pour le fou, c’est tous les jours fête",[était]


In [8]:
import re

# new dataframe with only one word masked per observation - index indicates old observation 
new_rows = []

for idx, row in df.iterrows():
    original_sentence = row['Masked']
    for verb in row['verbes_in_masked']:
        masked_sentence = re.sub(rf'\b{re.escape(verb)}\b', '<mask>', original_sentence)
        new_rows.append({
            'index': idx,  # Shared index
            'masked_sentence': masked_sentence,
            'word_list': row['Word_list'],  # Word candidates
            'original_masked_proverb': row['Masked'],  # Original masked proverb for reference
            'ground_truth': row['Proverb']
        })

# new dataframe
masked_df = pd.DataFrame(new_rows)
print(masked_df.head(10))

    index                        masked_sentence         word_list  \
0       0    <mask> beau mentir qui part de loin  [vient, revient]   
1       0         a beau <mask> qui part de loin  [vient, revient]   
2       0       a beau mentir qui <mask> de loin  [vient, revient]   
3       1   <mask> beau dormir qui vient de loin  [partir, mentir]   
4       1        a beau <mask> qui vient de loin  [partir, mentir]   
5       1       a beau dormir qui <mask> de loin  [partir, mentir]   
6       2            l’occasion <mask> le larron  [fait, occasion]   
7       3           endors-toi, le ciel t’aidera     [bouge, aide]   
8       3           endors-toi, le ciel t’aidera     [bouge, aide]   
9       3           endors-toi, le ciel t’<mask>     [bouge, aide]   
10      4             <mask>-toi, le ciel t’aura    [aidera, aide]   
11      4          aide<mask>toi, le ciel t’aura    [aidera, aide]   
12      4             aide-toi, le ciel t’<mask>    [aidera, aide]   
13      5      ce qu

## Section 3 - Modèle et code pour corriger un proverbe

Expliquez ici comment vous procédez pour choisir la meilleure version parmi les proverbes modifiés. 

In [9]:
from transformers import CamembertModel, CamembertTokenizer

tokenizer = CamembertTokenizer.from_pretrained("camembert/camembert-base-oscar-4gb")
camembert = CamembertModel.from_pretrained("camembert/camembert-base-oscar-4gb")

camembert_fill_mask  = pipeline("fill-mask", model="camembert/camembert-base-oscar-4gb", tokenizer="camembert/camembert-base-oscar-4gb")




Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [10]:
# Prediction function for masked sentences
def make_predictions(row):
    try:
        return camembert_fill_mask(row['masked_sentence'], targets=row['word_list'])
    except Exception as e:
        # return error message in case something goes wrong
        return str(e)

# Applying it
masked_df['predictions'] = masked_df.apply(make_predictions, axis=1)


The specified target token `revient` does not exist in the model vocabulary. Replacing with `▁revient`.
The specified target token `revient` does not exist in the model vocabulary. Replacing with `▁revient`.
The specified target token `revient` does not exist in the model vocabulary. Replacing with `▁revient`.
The specified target token `partir` does not exist in the model vocabulary. Replacing with `▁partir`.
The specified target token `mentir` does not exist in the model vocabulary. Replacing with `▁mentir`.
The specified target token `partir` does not exist in the model vocabulary. Replacing with `▁partir`.
The specified target token `mentir` does not exist in the model vocabulary. Replacing with `▁mentir`.
The specified target token `partir` does not exist in the model vocabulary. Replacing with `▁partir`.
The specified target token `mentir` does not exist in the model vocabulary. Replacing with `▁mentir`.
The specified target token `bouge` does not exist in the model vocabulary. R

In [11]:
pd.set_option('display.max_colwidth', None)  # Show full content of cells
print(masked_df['predictions'].head(10))

0                                  [{'score': 5.756229626285858e-08, 'token': 2151, 'token_str': 'revient', 'sequence': 'revient beau mentir qui part de loin'}, {'score': 4.6352996996290585e-09, 'token': 17068, 'token_str': 'vient', 'sequence': 'vient beau mentir qui part de loin'}]
1                                            [{'score': 1.6705618691048585e-05, 'token': 2151, 'token_str': 'revient', 'sequence': 'a beau revient qui part de loin'}, {'score': 4.7252122215013515e-08, 'token': 17068, 'token_str': 'vient', 'sequence': 'a beauvient qui part de loin'}]
2                                            [{'score': 0.09831991046667099, 'token': 2151, 'token_str': 'revient', 'sequence': 'a beau mentir qui revient de loin'}, {'score': 8.770224724230502e-08, 'token': 17068, 'token_str': 'vient', 'sequence': 'a beau mentir quivient de loin'}]
3                                [{'score': 3.5203133847971912e-06, 'token': 350, 'token_str': 'partir', 'sequence': 'partir beau dormir qui vient d

In [29]:
# extract the word with the highest score
def get_highest_score_prediction(predictions):
    if isinstance(predictions, list) and len(predictions) > 0:
        best_prediction = max(predictions, key=lambda x: x['score'])
        return best_prediction['token_str']  # Return token (i.e. word)
    return None  # none if pred are empty or invalid

masked_df['final_prediction'] = masked_df['predictions'].apply(get_highest_score_prediction)

                      ground_truth                       masked_sentence  \
0  a beau mentir qui vient de loin   <mask> beau mentir qui part de loin   
1  a beau mentir qui vient de loin        a beau <mask> qui part de loin   
2  a beau mentir qui vient de loin      a beau mentir qui <mask> de loin   
3  a beau mentir qui vient de loin  <mask> beau dormir qui vient de loin   
4  a beau mentir qui vient de loin       a beau <mask> qui vient de loin   
5  a beau mentir qui vient de loin      a beau dormir qui <mask> de loin   
6        l’occasion fait le larron           l’occasion <mask> le larron   
7       aide-toi, le ciel t’aidera          endors-toi, le ciel t’aidera   
8       aide-toi, le ciel t’aidera          endors-toi, le ciel t’aidera   
9       aide-toi, le ciel t’aidera          endors-toi, le ciel t’<mask>   

          word_list final_prediction  \
0  [vient, revient]          revient   
1  [vient, revient]          revient   
2  [vient, revient]          revient   
3  

In [13]:
# Reorder cols for better readability
masked_df = masked_df[['ground_truth', 'masked_sentence', 'word_list', 'final_prediction', 'predictions', 'original_masked_proverb', 'index']]
print(masked_df.head(10))

                      ground_truth                       masked_sentence  \
0  a beau mentir qui vient de loin   <mask> beau mentir qui part de loin   
1  a beau mentir qui vient de loin        a beau <mask> qui part de loin   
2  a beau mentir qui vient de loin      a beau mentir qui <mask> de loin   
3  a beau mentir qui vient de loin  <mask> beau dormir qui vient de loin   
4  a beau mentir qui vient de loin       a beau <mask> qui vient de loin   
5  a beau mentir qui vient de loin      a beau dormir qui <mask> de loin   
6        l’occasion fait le larron           l’occasion <mask> le larron   
7       aide-toi, le ciel t’aidera          endors-toi, le ciel t’aidera   
8       aide-toi, le ciel t’aidera          endors-toi, le ciel t’aidera   
9       aide-toi, le ciel t’aidera          endors-toi, le ciel t’<mask>   

          word_list final_prediction  \
0  [vient, revient]          revient   
1  [vient, revient]          revient   
2  [vient, revient]          revient   
3  

In [14]:
# get highest scoring prediction for each index group
def get_highest_score_row(group):
    # only valid predictions
    valid_predictions = group[group['predictions'].apply(lambda x: isinstance(x, list))]
    if valid_predictions.empty:
        return None  # if no valid prediction exists

    # max score row
    best_row = valid_predictions.loc[valid_predictions['predictions'].apply(
        lambda preds: max([p['score'] for p in preds] if isinstance(preds, list) else [0])
    ).idxmax()]
    return best_row

final_df = masked_df.groupby('index').apply(get_highest_score_row).reset_index(drop=True)
print(final_df.head())


                      ground_truth                   masked_sentence  \
0  a beau mentir qui vient de loin  a beau mentir qui <mask> de loin   
1  a beau mentir qui vient de loin   a beau <mask> qui vient de loin   
2        l’occasion fait le larron       l’occasion <mask> le larron   
3       aide-toi, le ciel t’aidera      endors-toi, le ciel t’<mask>   
4       aide-toi, le ciel t’aidera        aide-toi, le ciel t’<mask>   

          word_list final_prediction  \
0  [vient, revient]          revient   
1  [partir, mentir]           partir   
2  [fait, occasion]         occasion   
3     [bouge, aide]            bouge   
4    [aidera, aide]           aidera   

                                                                                                                                                                                                                                       predictions  \
0   [{'score': 0.09831991046667099, 'token': 2151, 'token_str': 'revient', 'sequ

/var/folders/cr/1qy7d39n5t728w232c3qvq8m0000gn/T/ipykernel_34520/2929172703.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  final_df = masked_df.groupby('index').apply(get_highest_score_row).reset_index(drop=True)


In [27]:
# construct the predicted proverb
def construct_predicted_proverb(row):
    if pd.isna(row['final_prediction']) or '<mask>' not in row['masked_sentence']:
        return 'CHECK ME AGAIN' # Return as is if no valid prediction or no mask present
    return row['masked_sentence'].replace('<mask>', row['final_prediction'])

# Create predicted proverb column
final_df['predicted_proverb'] = final_df.apply(construct_predicted_proverb, axis=1)

print(final_df[['ground_truth', 'predicted_proverb']])


                                                             ground_truth  \
0                                         a beau mentir qui vient de loin   
1                                         a beau mentir qui vient de loin   
2                                               l’occasion fait le larron   
3                                              aide-toi, le ciel t’aidera   
4                                              aide-toi, le ciel t’aidera   
5                                         ce que femme veut, dieu le veut   
6                                         ce que femme veut, dieu le veut   
7                                       bien mal acquis ne profite jamais   
8                                  bon ouvrier ne querelle pas ses outils   
9                                  pour le fou, c’est tous les jours fête   
10                                               dire et faire, sont deux   
11                                               dire et faire, sont deux   

In [24]:
# Calculate Prediction Accuracy
final_df['is_correct'] = final_df['predicted_proverb'] == df['Proverb']
accuracy_score = final_df['is_correct'].mean() * 100

print(f"Accuracy: {accuracy_score:.2f}%")

num_correct = final_df['is_correct'].sum()
total_observations = len(final_df)
print(f"Correct Predictions: {num_correct}/{total_observations}")



Accuracy: 65.38%
Correct Predictions: 17/26


In [23]:
print(results)

[{'score': 0.17374156415462494, 'token': 6256, 'token_str': 'Très', 'sequence': 'Très peu, chasse beaucoup de maladies'}, {'score': 0.17374156415462494, 'token': 6256, 'token_str': 'Très', 'sequence': 'Très peu, chasse beaucoup de maladies'}, {'score': 0.17374156415462494, 'token': 6256, 'token_str': 'Très', 'sequence': 'Très peu, chasse beaucoup de maladies'}, {'score': 0.17374156415462494, 'token': 6256, 'token_str': 'Très', 'sequence': 'Très peu, chasse beaucoup de maladies'}, {'score': 0.17374156415462494, 'token': 6256, 'token_str': 'Très', 'sequence': 'Très peu, chasse beaucoup de maladies'}, {'score': 0.17374156415462494, 'token': 6256, 'token_str': 'Très', 'sequence': 'Très peu, chasse beaucoup de maladies'}, {'score': 0.17374156415462494, 'token': 6256, 'token_str': 'Très', 'sequence': 'Très peu, chasse beaucoup de maladies'}, {'score': 0.17374156415462494, 'token': 6256, 'token_str': 'Très', 'sequence': 'Très peu, chasse beaucoup de maladies'}, {'score': 0.17374156415462494, 

## Section 4 - Expérimentations et analyse de résultats


L'approche générale que nous avons choisie pour la tâche 3 consiste en 5 parties :

#### 1. POS Tagging
Nous avons utilisé le modèle gilf/french-camembert-postag-model pour effectuer le POS Tagging. Parfois, le modèle retourne le mot correct qui doit être remplacé, parfois il retourne plus de verbes (par exemple, dans le premier proverbe masqué "a beau mentir qui vient de loin", le modèle retourne trois verbes : "a", "mentir" et "vient"). Tous les verbes sont conservés pour être testés dans l'étape de prédiction du verbe masqué. Le modèle réussit à trouver la plupart des verbes, mais dans certains cas, il identifie d'autres termes incorrectement. Par exemple, dans le proverbe "aide-toi, le ciel t'aura", il identifie le trait d'union comme un verbe. Cette erreur ne devrait pas être trop importante puisque le deuxième modèle devrait identifier que "aideaidetoi" (le trait d'union remplacé par "aide") n'a aucun sens et donnera un très petit score à ce terme.

#### 2. Création d'un nouveau DataFrame pandas avec une ligne par verbe détecté (par POS Tagging) et un verbe masqué
Étant donné que nous savons qu'un seul mot est modifié par ligne, nous avons choisi de créer un nouveau DataFrame pandas qui contient une ligne par verbe détecté (par ligne originale). Ainsi, si nous prenons le premier exemple, le nouveau DataFrame a 3 lignes pour le proverbe "a beau mentir qui vient de loin". Dans chaque ligne, un des verbes est masqué :

a) <mask> beau mentir qui vient de loin

b) a beau <mask> qui vient de loin 

c) a beau mentir qui <mask> de loin

#### 3. Prédiction du verbe masqué pour chaque ligne
Nous utilisons le modèle camembert/camembert-base-oscar-4gb pour prédire le mot masqué en utilisant la liste des verbes donnés. Le modèle retourne des scores pour chaque mot candidat. Pour l'exemple précédent, 2 scores par ligne sont retournés, donc au total 6 scores pour le premier proverbe avec les trois mots masqués. Nous choisissons entre les 2 scores par verbe masqué par ligne le verbe avec le score le plus grand.

Pour faire l'inférence, nous utilisons le pipeline de "fill-mask" de Transformers. Le modèle retourné permet de spécifier des _targets_, c'est-à-dire les mots à utiliser pour le masque. Des avertissements indiquent que tous les mots ne font pas partie du vocabulaire du modèle. Il semblerait que cela soit dû au fait que la tokenisation des _targets_ se fait avec [BERT au lieu de celle fournie par le modèle](https://l.messenger.com/l.php?u=https%3A%2F%2Fgithub.com%2Fhuggingface%2Ftransformers%2Fissues%2F19379&h=AT2TNkJEfARZGqrsYbqtytAamnLWh_6ZKRVP9V-uCYlQVYUERuvbbckqWzDEBJ8-KO29FdBrII-Jd5hwxzNaBfXBWu_ZQ5NMnPo1RSq9FGIVo-_9BL9RdcADmbFC7bJwIiyRn0mQKUp0EUI). Nous avons donc testé en fournissant les termes préfixés par "_" qui semble être le format du vocabulaire de Camembert. Ainsi, seulement un mot, "_pourri", ne se trouve pas dans le vocabulaire. Cependant, il semblerait que le terme utilisé pour remplacer le mot non préfixé de "_" soit efficace puisque le changement n'a pas permis d'améliorer les résultats. Au contraire, les performances étaient inférieures en préfixant les mots, puisque le mot "_pourrit" qui n'est pas dans le vocabulaire est remplacé par "_pour", ce qui n'a pas la même signification et cause le modèle de se tromper sur le proverbe avec ce terme.

#### 4. Sélection du score le plus élevé pour choisir le mot masqué et le remplacer
Dans la partie suivante du code, nous comparons les scores entre les lignes qui correspondent au même proverbe (nous avons introduit un indice auparavant). Encore une fois, nous choisissons le mot avec le score le plus élevé pour le remplacer. Ainsi, ici, nous choisissons le mot qui est remplacé basé sur le score. Nous suivons l'hypothèse que le bon mot candidat sera meilleur dans le bon contexte correct que ce mot dans les autres contextes ou l'autre mot dans tous les contextes.

#### 5. Comparaison de la prédiction avec la vérité
Finalement, nous remplaçons le token <mask> par le mot prédit dans le proverbe et le comparons avec le vrai proverbe. Le modèle réussit à prédire correctement 17 proverbes sur 26, ce qui correspond à une précision de 65,38 %.
